# PRODUCTIVIZAR UN MODELO - EJEMPLO 01

* Vamos a crear diferentes funciones que vamos a exponer vía API sobre el conjunto de Gapminder.
* Estas funciones nos permitirán procesar el conjunto y obtener diferentes resultados.
* Expondremos las funciones.
* Y obtendremos diferentes resultados. No son modelos, son procesados a partir de unas entradas que nos llegan a través del API.

### Importamos Librerías

In [ ]:
import pandas as pd
from plotnine import *
from io import BytesIO
from flask import *

import warnings
warnings.filterwarnings("ignore")

### Inicializamos la aplicación de Flask

In [ ]:
app = Flask(__name__)

### Modelo

In [ ]:
gapminder_df = pd.read_csv('../data/gapminder.csv')

In [ ]:
gapminder_df.head()

In [ ]:
gapminder_df.shape

## Apificamos diferentes llamadas al conjunto "gapminder"

In [ ]:
# Primera función - continent_pop
@app.route('/continent_pop', methods=['POST'])
def continent_pop():
    # Consigo los datos de la petición
    data = request.get_json(force=True)

    continent = data['continent']
    pop = data['pop']
    
    result = gapminder_df.loc[(gapminder_df['continent']==continent) & 
                       (gapminder_df['pop'] >= pop) & 
                       (gapminder_df['year']== 2007)]

    # La predicción la convierto a un JSON
    return jsonify(result.to_json(orient='records'))

# Segunda función - pais_life
@app.route('/pais_life')
def pais_life():
    # Consigo los datos de la petición
    data = request.get_json(force=True)

    pais = data['country']
    title = data['title']
    
    result = gapminder_df.loc[(gapminder_df['country']==pais)]
    
    chart = (ggplot(result)
             + aes(x='year', y='lifeExp')
             + geom_line()
             + labs( title = "Evolución Esperanza de Vida - " + title, x = "Año", y = "Esperanza de Vida")
             + theme_bw()
            )
    
    # renderiza el gráfico como una imagen PNG
    img = BytesIO()
    chart.save(img, format="png")
    
    # Regresa el archivo PNG al cliente
    img.seek(0)
    return send_file(img, mimetype="image/png")
    

In [ ]:
# Levanto la aplicación en el puerto 8080 para
if __name__ == '__main__':
    app.run(port=8080)

---
---

### Lanzo una petición al modelo

In [ ]:
### Incluir estas líneas en un notebook diferente.

#------------------------ continent_pop
import requests

data = {
    "continent": "Europe",
    "pop": 40e6,
}

### Hago la petición y obtengo la respuesta

response = requests.post("http://localhost:8080/continent_pop", json=data)
prediction = response.json()
prediction

#-- El resultado de la predicción es:
import pandas as pd
pred_df = pd.read_json(prediction)
pred_df


#------------------------ pais_life
import requests

data = {
    "country": "Spain",
    "title": "España",
}

### Hago la petición y obtengo la respuesta
respuesta = requests.get("http://localhost:8080/pais_life", json=data)

# Guarda el gráfico en un archivo PNG
with open("Gráfico Salida.png", "wb") as f:
    f.write(respuesta.content)

# Guarda el gráfico en un archivo PNG
with open("Esperanza_Vida_.png", "wb") as f:
    f.write(respuesta.content)